## Coursera Capstone - Week 4 & 5

In [3]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


In [17]:
import pandas as pd
import numpy as np
import requests 
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

!pip install geopy
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

! pip install folium==0.5.0
import folium 

import matplotlib.pyplot as plt
import pylab as pl
%matplotlib inline

!pip install bs4
!pip install lxml
!pip install html5lib

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/util.py:25: CryptographyDeprecationWarning: int_from_bytes is deprecated, use int.from_bytes instead
  from cryptography.utils import int_from_bytes
/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/secretstorage/dhcrypto.py:16: Cryptograp

# Introduction/Business Problem

Company ABC is looking to open it's first US office, in a major metropoliatain area.

The company needs a highly skilled and happy workforce. 

The company's workforce love art galleries, gyms, coffee shops and wine bars. This frequency of these locations will be the proxy for a happy workforce.

We will leverage the foursquare API to rank these items by city. 

We will build a regression model to help the company decide where to open the US office. There should be a link between the number of art galleries, gyms, coffee shops and wine bars (the independent variables), and per captia income (the dependent variable) which we will use as a proxy for a highly skilled workforce.

We will test multiple models to see which has the strongest link with capita income - art galleries, gyms, coffee shops and wine bars?

# Data

In [1]:
# Describe the data that you will be using to solve the problem or execute your idea. Remember that you will need to use the Foursquare location data to solve the problem or execute your idea. 
#You can absolutely use other datasets in combination with the Foursquare location data. 
#So make sure that you provide adequate explanation and discussion, with examples, of the data that you will be using, even if it is only Foursquare location data.

We wil leverage the foursquare API for the top 20 cities by per capita income.

In order to do this we will pull the city lat long from from geopy.geocoders Nominatim module, to convert an address into latitude and longitude values (see section 3 data)

We will pull back data on the number of art galleries, gyms, coffee shops and wine bars by city.

## 1. Average Income by Cities (top 10)

Average Per Capita income by US City

https://en.wikipedia.org/wiki/List_of_United_States_metropolitan_areas_by_per_capita_income

Source: US 2010 Census

### 1.1 Example data

In [7]:
url2= 'https://en.wikipedia.org/wiki/List_of_United_States_metropolitan_areas_by_per_capita_income'
html_content = requests.get(url2).text

dfs = pd.read_html(html_content)

df = dfs[2]
df.head(20)

,Rank,Metropolitan statistical area,Population,Per capitaincome
0,1,"Washington-Arlington-Alexandria, D.C-Virginia-...",5949178,"$47,411"
1,2,"San Jose-Santa Clara-Sunnyvale, California MSA",1918944,"$40,392"
2,3,"Seattle-Tacoma-Bellevue, Washington MSA",3611644,"$39,322"
3,4,"San Francisco-Oakland-Hayward, California MSA",4122177,"$38,355"
4,5,"Boston–Worcester–Lawrence, Massachusetts–New H...",5819100,"$37,311"
5,6,"Honolulu, Hawaii MSA",921000,"$36,339"
6,7,"Minneapolis-St. Paul-Bloomington, Minnesota MSA",3478415,"$35,388"
7,8,"Hartford, Connecticut MSA",1183110,"$34,310"
8,9,"Denver-Aurora-Lakewood, Colorado MSA",2871068,"$32,399"
9,10,"Portland-Vancouver-Hillsboro, Oregon MSA",2345318,"$31,377"


## 2. Lat Long data

We will use geopy.geocoders Nominatim module, to convert each city into latitude and longitude values

### 2.1 Example data

In [10]:

address = 'Washington, DC'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Washington, DC are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Washington, DC are 38.8950368, -77.0365427.


## 3. Foursquare location data

### 3.1 Example data

In [9]:
CLIENT_ID = 'YV4QTJAQ3F1OGZG2AFPTE5SEPY02PS3CLHHK35VXRDDQE3WZ' # your Foursquare ID
CLIENT_SECRET = 'JSWHMGPW210QWZQSYEBQEDLZK3OS2XXZQ0KA22O4AYBNLXI4' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: YV4QTJAQ3F1OGZG2AFPTE5SEPY02PS3CLHHK35VXRDDQE3WZ
CLIENT_SECRET:JSWHMGPW210QWZQSYEBQEDLZK3OS2XXZQ0KA22O4AYBNLXI4


In [12]:
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?client_id=YV4QTJAQ3F1OGZG2AFPTE5SEPY02PS3CLHHK35VXRDDQE3WZ&client_secret=JSWHMGPW210QWZQSYEBQEDLZK3OS2XXZQ0KA22O4AYBNLXI4&ll=38.8950368,-77.0365427&v=20180605&radius=500&limit=100'

In [13]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '606883d604959b5a1f59bc3d'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': '$-$$$$', 'key': 'price'},
    {'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Northwest Washington',
  'headerFullLocation': 'Northwest Washington, Washington',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 46,
  'suggestedBounds': {'ne': {'lat': 38.8995368045, 'lng': -77.03077163914982},
   'sw': {'lat': 38.8905367955, 'lng': -77.04231376085018}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4bb9e67c1261d13aeef4e998',
       'name': 'South Lawn',
       'location': {'address': '1600 Pennsylvania Ave NW',
        'lat': 38.89672777385296,
        'lng': -77.0366048812866,
        'label

In [15]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [20]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/opt/conda/envs/Python-3.7-main/lib/python3.7/site-packages/ipykernel/__main__.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  app.launch_new_instance()


,categories,lat,lng
0,Garden,38.896728,-77.036605
1,Park,38.894229,-77.036605
2,Tour Provider,38.896508,-77.033837
3,Hotel,38.897037,-77.033322
4,Garden,38.897472,-77.037088
5,Hotel,38.897215,-77.032541
6,Monument / Landmark,38.895942,-77.038711
7,American Restaurant,38.897954,-77.033270
8,Garden,38.895820,-77.036881
9,American Restaurant,38.896507,-77.033040
